In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Here I link my database in my personal Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pathlib
data_dir = pathlib.Path("/content/drive/MyDrive/dataset")


In [ ]:
batch_size = 16
img_height = 300
img_width = 300

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 7365 files belonging to 10 classes.
Using 5892 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 7365 files belonging to 10 classes.
Using 1473 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['apfelstrudel', 'bienenstich', 'donauwelle', 'eierschecke', 'frankfurter kranz', 'kaiserschmarn', 'kalter hund', 'sachertorte', 'schwarzwalder kirschtorte', 'streuselkuchen']


In [ ]:
###This is supposed to be used only once, to eliminate all non-pictures or non-supported files of the database

#from pathlib import Path
#import imghdr
#import os

#data_dir = "/content/drive/MyDrive/dataset"
#image_extensions = [".png", ".jpg"]  # add there all your images file extensions

#img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
#for filepath in Path(data_dir).rglob("*"):
#    if filepath.suffix.lower() in image_extensions:
#        img_type = imghdr.what(filepath)
#        if img_type is None:
#            print(f"{filepath} is not an image")
#            os.remove(filepath)
#        elif img_type not in img_type_accepted_by_tf:
#            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
#            os.remove(filepath)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(16, 300, 300, 3)
(16,)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
num_classes = len(class_names)


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(48, activation='relu'),
  layers.Dense(num_classes)
])



In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 300, 300, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 300, 300, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 300, 300, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 150, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                  

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=5)
epochs = 50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[early_stopping]
)

Epoch 1/50
369/369 [==============================] - 1219s 689ms/step - loss: 2.0538 - accuracy: 0.2071 - val_loss: 1.8085 - val_accuracy: 0.2498
Epoch 2/50
369/369 [==============================] - 15s 41ms/step - loss: 1.8126 - accuracy: 0.2899 - val_loss: 1.6957 - val_accuracy: 0.3245
Epoch 3/50
369/369 [==============================] - 15s 40ms/step - loss: 1.6626 - accuracy: 0.3888 - val_loss: 1.5402 - val_accuracy: 0.4406
Epoch 4/50
369/369 [==============================] - 15s 41ms/step - loss: 1.5360 - accuracy: 0.4420 - val_loss: 1.4180 - val_accuracy: 0.4800
Epoch 5/50
369/369 [==============================] - 15s 41ms/step - loss: 1.4348 - accuracy: 0.4810 - val_loss: 1.4091 - val_accuracy: 0.4793
Epoch 6/50
369/369 [==============================] - 15s 41ms/step - loss: 1.3908 - accuracy: 0.5012 - val_loss: 1.4783 - val_accuracy: 0.4650
Epoch 7/50
369/369 [==============================] - 15s 41ms/step - loss: 1.3418 - accuracy: 0.5227 - val_loss: 1.2702 - val_accura

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(val_acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predict on new data

In [ ]:
tf.keras.backend.clear_session()
images = ["https://cdn.lowcarb-community.de/2015/11/Donauwelle.jpg","https://karls-shop.b-cdn.net/media/49/29/a6/1619182636/kuchen-bienenstich-szene1.jpg","http://www.ankerbrot.at/io/produkt_img/2297.jpg","https://rezepte-silkeswelt.de/wp-content/uploads/2020/02/Pudding-Streuselkuchen1-scaled-e1629386746610.jpg"]

for BS_url in images: 
  BS_path = tf.keras.utils.get_file(origin=BS_url)


  img = tf.keras.utils.load_img(
      BS_path, target_size=(img_height, img_width)
  )
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
  )


Saving normal model and saving as TensorFlow Lite for later use in Android

In [ ]:
model.save('model.h5')


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)